# プロジェクト - 航空会社AIアシスタント

航空会社のAIカスタマーサポートアシスタントを作成するために学んだことをまとめます

In [ ]:
# 輸入

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# 初期化

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("Openai APIキーが設定されていません")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# この関数は、最新のグラデーションアップデートを利用しているため、私のビデオの機能よりもかなりシンプルに見えます。

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## ツール

ツールは、フロンティアLLMSが提供する非常に強力な機能です。

ツールを使用すると、関数を記述し、LLMにその機能をその応答の一部として呼び出すことができます。

ほとんど不気味に聞こえます。マシンでコードを実行する力を与えていますか？

まあ、ちょっと。

In [ ]:
# 役立つ機能を作成することから始めましょう

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
get_ticket_price("London")

In [ ]:
# 私たちの機能を説明するために必要な特定の辞書構造があります。

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# これはツールのリストに含まれています。

tools = [{"type": "function", "function": price_function}]

## Openaiを取得してツールを使用します

Openaiが「私たちのツールを呼び出す」ことを許可するための厄介なものがいくつかあります

私たちが実際に行っていることは、LLMにツールを実行することを望んでいることを知らせる機会を与えることです。

新しいチャット関数がどのように見えるかは次のとおりです。

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# その関数をhandle_tool_call：書く必要があります：

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

# マルチモーダルに行きましょう!!

GPT-4oの背後にある画像生成モデルであるDall-E-3を使用して、画像を作成できます。

これをアーティストと呼ばれる関数に入れましょう。

###価格アラート：画像を生成するたびに約4セントかかります - 画像に夢中にならないでください！

In [ ]:
# 画像を処理するためのいくつかのインポート

import base64
from io import BytesIO
from PIL import Image

In [ ]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("New York City")
display(image)

## オーディオ（注 - このコースのオーディオはオプションです - 問題が発生した場合は、オーディオをスキップしてください！）

そして、Openaiの音声モデルを使用してオーディオを生成する機能トーカーを作りましょう

###トラブルシューティングオーディオの問題

以下にこのコードを実行するのに問題がある場合（FilenotFoundエラーや欠落パッケージの警告など）、非常に人気のあるオーディオユーティリティであるFFMPEGをインストールする必要がある場合があります。

** PCユーザー向け**

詳細な指示は[here](https://chatgpt.com/share/6724efee-6b0c-8012-ac5e-72e2e3885905)と要約手順です。

1.公式ウェブサイトからFFMPEGをダウンロード：https：//ffmpeg.org/download.html

2.ダウンロードしたファイルをコンピューターの場所に抽出します（例： `c：\ ffmpeg`）

3. ffmpeg binフォルダーをシステムパスに追加します。
 - 「このPC」または「私のコンピュータ」を右クリックして、[プロパティ]を選択します
 - 「Advanced Systemstings」をクリックします
 - 「環境変数」をクリックします
 - 「システム変数」の下で、「パス」を見つけて編集します
-FFMPEGビンフォルダーへのパスで新しいエントリを追加します（例： `c：\ ffmpeg \ bin`）
 - コマンドプロンプトを再起動し、Jupyter Lab内でカーネルを行います - >カーネルを再起動して、変更を受け取ります

4.新しいコマンドプロンプトを開いて、これを実行して、それがインストールされていることを確認してください
`ffmpeg -version`

** Macユーザー向け**

1.ターミナルウィンドウでこれを実行し、指示に従ってこれを実行していない場合は、Homebrewをインストールしてください。  
`/bin/bash -c" $（curl -fssl https://raw.githubusercontent.com/homebrew/install/head/install.sh） "`

2。「brewインストールffmpeg」でffmpegをインストールします

3.「ffmpeg -version」でインストールを確認し、すべてが良い場合は、jupyter lab内でカーネルを行います - >カーネルを再起動して変更を受け取ります

私にメッセージを送るか、ed@edwarddonner.comに私にメールしてください。

## To check you now have ffmpeg and can access it here

Excecute the next cell to see if you get a version number. (Putting an exclamation mark before something in Jupyter Lab tells it to run it as a terminal command rather than python code).

これがうまくいかない場合は、Jupyterラボを実際に保存して閉じて、LLMS環境をアクティブにすることを覚えている新しいターミナルウィンドウ（MAC）またはAnacondaプロンプト（PC）から再度開始する必要がある場合があります。 This ensures you pick up ffmpeg.

And if that doesn't work, please contact me!

In [ ]:
!ffmpeg -version
!ffprobe -version
!ffplay -version

# Macユーザーの場合 - そしておそらく多くのPCユーザーも

このバージョンはあなたのために正常に動作するはずです。 Windowsユーザーにも機能する可能性がありますが、Tempファイルへのアクセス許可エラー書き込みが表示される場合があります。もしそうなら、次のセクションをご覧ください！

いつものように、問題がある場合は、私に連絡してください！ （オーディオ生成にあまり興味がない場合は、後のコードでAudio Talker（）にコメントすることもできます）

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # また、Onyxを合金に置き換えてみてください
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
talker("Well, hi there")

# Windowsユーザー（または上記の問題があるMacユーザー）の場合）

##最初に上記のMacバージョンを試してくださいが、Tempファイルへのアクセス許可エラー書き込みが表示された場合は、代わりにこのコードが機能するはずです。

学生のマーク・Mとパトリック・Hとクロードのコラボレーションは、これを解決しました！

以下は4つのバリエーションです - うまくいけば、そのうちの1つがPCで動作します。そうでない場合は、私にメッセージを送ってください！

そして、Macの人々のために - 以下の3つの作業も私のMacで作業します -  Macバージョンがあなたに問題を与えたなら、これらを試してください。

## PCバリエーション1

In [ ]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # 生成されたオーディオを再生します
    display(Audio(output_filename, autoplay=True))

talker("Well, hi there")

## PCバリエーション2

In [ ]:
import tempfile
import subprocess
from io import BytesIO
from pydub import AudioSegment
import time

def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(3) # 学生ドミニクは、これが必要であることを発見しました。また、PCで必要でないかどうかを確認するためにコメントしてみることもできます
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass
 
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # また、Onyxを合金に置き換えてみてください
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)

talker("Well hi there")

## PCバリエーション3

In [ ]:
import os
from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO

def talker(message):
    # Windows上の一時ファイルのカスタムディレクトリを設定します
    custom_temp_dir = os.path.expanduser("~/Documents/temp_audio")
    os.environ['TEMP'] = custom_temp_dir  # 必要に応じて「TMP」を使用することもできます
    
    # フォルダーが存在しない場合は、フォルダーを作成します
    if not os.path.exists(custom_temp_dir):
        os.makedirs(custom_temp_dir)
    
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # また、Onyxを合金に置き換えてみてください
        input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    play(audio)

talker("Well hi there")

## PCバリエーション4

###まったく異なるサウンドライブラリを試してみましょう

最初に次のセルを実行して新しいライブラリをインストールしてから、その下のセルを試してください。

In [ ]:
!pip install simpleaudio

In [ ]:
from pydub import AudioSegment
from io import BytesIO
import tempfile
import os
import simpleaudio as sa

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # また、Onyxを合金に置き換えてみてください
        input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    # 書き込み許可があるフォルダーに一時ファイルを作成する
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False, dir=os.path.expanduser("~/Documents")) as temp_audio_file:
        temp_file_name = temp_audio_file.name
        audio.export(temp_file_name, format="wav")
    
    # Simpleaudioを使用してオーディオを読み込んで再生します
    wave_obj = sa.WaveObject.from_wave_file(temp_file_name)
    play_obj = wave_obj.play()
    play_obj.wait_done()  # 再生が終了するのを待ちます

    # その後、一時ファイルをクリーンアップします
    os.remove(temp_file_name)
    
talker("Well hi there")

## PCユーザー - これらの4つのバリエーションのいずれも機能しなかった場合！

私に連絡してください。申し訳ありませんが、これが問題を引き起こしています！私たちはそれを理解します。

または、PCからオーディオを再生することは、このコースでは非常に批判的ではありません。今のところ画像生成に集中してオーディオをスキップするか、後で戻ってくることができます。

# エージェントフレームワーク

「エージェントAI」という用語とエージェント化は、次のような多くの手法を指す傘の用語です。

1.複数のLLMが特別なタスクを実行して、複雑な問題をより小さなステップに分割する
2。LLMSがツールを使用して追加の機能を提供する能力
3。エージェントが協力できる「エージェント環境」
4. LLMはプランナーとして機能し、スペシャリストのために大きなタスクを小さなタスクに分割することができます
5。自治 /代理店を持っているエージェントの概念、プロンプトに単に応答するだけで - メモリなど

ここでは1と2を示していますが、それほどではありません。

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # 今のところオーディオをスキップしたい場合は、次の行をコメントまたは削除してください。
    talker(reply)
    
    return history, image

In [ ]:
# プリセットチャットインターフェイスを使用していないため、より複雑なグラデーションコード！
# inbrowserを渡す=最後の行でtrueを使用すると、グラデーションウィンドウがすぐにポップアップされます。

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

# 演習とビジネスアプリケーション

より多くのツールを追加する - おそらく実際にフライトを予約するシミュレーション。学生がこれを行って、コミュニティの寄付フォルダーでその例を提供しました。

次：これを取り、それをあなたのビジネスに適用してください。作業のアクティビティを実行できるツールを使用して、マルチモーダルAIアシスタントを作成します。カスタマーサポートアシスタント？新しい従業員のオンボーディングアシスタント？たくさんの可能性！また、別のノートブックの週の終わりのエクササイズを参照してください。

<テーブルスタイル= "マージン：0;テキストアライグ：左;">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ありがとうyou.jpg" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃090;">特別なリクエストがあります</h2>
            <Span style = "color：＃090;">
                私の編集者は、学生がこのコースをUdemyで評価するときに大きな違いをもたらすと言っています。これは、Udemyが他の人に見せるかどうかを決定する主な方法の1つです。あなたがこれを評価するのに少し時間がかかるなら、私はとても感謝しています！とにかく - いつでも支援できる場合は、常にed@edwarddonner.comで私に連絡してください。
            </span>
        </td>
    </tr>
</table>